# Custom Policies Workshop

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/batmanvane/molt-shield/blob/main/notebooks/custom_policies_workshop.ipynb)

**Create and manage security policies** for different types of simulation data.

This notebook shows engineers how to:
- Analyze different XML schemas (CFD, FEA, thermal)
- Auto-detect sensitive elements
- Create custom tag shadow mappings
- Test and validate policies

# ============================================================
# SETUP: Install dependencies (run this FIRST)
# ============================================================
# These packages are NOT pre-installed in Google Colab

!pip install -q lxml pydantic pyyaml

print("✓ Dependencies installed: lxml, pydantic, pyyaml")

# Clone repository
!git clone https://github.com/batmanvane/molt-shield.git /content/molt-shield

import sys
sys.path.insert(0, '/content/molt-shield/src')

import os
os.chdir('/content/molt-shield')

print("✓ Environment ready!")

In [ ]:
# ============================================================
# SETUP: Install dependencies (run this FIRST)
# ============================================================
# These packages are NOT pre-installed in Google Colab

!pip install -q lxml pydantic pyyaml

print("✓ Dependencies installed: lxml, pydantic, pyyaml")

# Clone repository
!git clone https://github.com/batmanvane/molt-shield.git /content/molt-shield

import sys
sys.path.insert(0, '/content/molt-shield/src')

import os
os.chdir('/content/molt-shield')

print("✓ Environment ready!")

## 2. Sample Data: Different Simulation Types

Let's create sample XML files for different engineering simulation types.

In [ ]:
# Create sample data for different simulation types

# CFD Simulation (Computational Fluid Dynamics)
CFD_XML = """<?xml version="1.0" encoding="UTF-8"?>
<cfd_simulation>
    <metadata>
        <project>CFD-2024-WIND-TURBINE</project>
        <software>Ansys Fluent</software>
    </metadata>
    <domain>
        <velocity_inlet>15.5</velocity_inlet>
        <pressure_outlet>101325</pressure_outlet>
        <temperature>300.0</temperature>
    </domain>
    <results>
        <point id="p1">
            <velocity_magnitude>12.3</velocity_magnitude>
            <pressure>101500</pressure>
            <turbulence_intensity>0.05</turbulence_intensity>
        </point>
        <point id="p2">
            <velocity_magnitude>14.7</velocity_magnitude>
            <pressure>101200</pressure>
            <turbulence_intensity>0.08</turbulence_intensity>
        </point>
    </results>
</cfd_simulation>"""

# FEA (Finite Element Analysis)
FEA_XML = """<?xml version="1.0" encoding="UTF-8"?>
<fea_analysis>
    <metadata>
        <project>BRIDGE-LOAD-2024</project>
        <software>ANSYS Mechanical</software>
    </metadata>
    <loads>
        <force id="f1">
            <magnitude>500000</magnitude>
            <direction x="0" y="-1" z="0"/>
        </force>
    </loads>
    <results>
        <node id="n1">
            <displacement_x>0.025</displacement_x>
            <displacement_y>-0.015</displacement_y>
            <stress_von_mises>250e6</stress_von_mises>
        </node>
    </results>
</fea_analysis>"""

# Thermal Analysis
THERMAL_XML = """<?xml version="1.0" encoding="UTF-8"?>
<thermal_analysis>
    <metadata>
        <project>ELECTRONIC-COOLING</project>
    </metadata>
    <boundary_conditions>
        <heat_flux>5000</heat_flux>
        <ambient_temp>25.0</ambient_temp>
        <convection_coeff>10.5</convection_coeff>
    </boundary_conditions>
    <results>
        <component id="chip">
            <temperature>85.3</temperature>
            <heat_generation>15.5</heat_generation>
            <thermal_resistance>0.02</thermal_resistance>
        </component>
    </results>
</thermal_analysis>"""

# Save samples
os.makedirs('sample_data', exist_ok=True)
with open('sample_data/cfd_simulation.xml', 'w') as f:
    f.write(CFD_XML)
with open('sample_data/fea_analysis.xml', 'w') as f:
    f.write(FEA_XML)
with open('sample_data/thermal_analysis.xml', 'w') as f:
    f.write(THERMAL_XML)

print("✓ Created sample data files:")
for f in os.listdir('sample_data'):
    print(f"  - sample_data/{f}")

## 3. Analyze XML Schema

Let's automatically detect tags and values to understand what needs to be protected.

In [ ]:
from lxml import etree
from collections import defaultdict

def analyze_xml(xml_string):
    """Analyze XML structure and identify numeric values."""
    tree = etree.fromstring(xml_string.encode())
    
    tags_with_values = defaultdict(list)
    tags_with_attributes = defaultdict(list)
    
    for elem in tree.iter():
        if elem.text and elem.text.strip():
            tags_with_values[elem.tag].append(elem.text.strip())
        for attr, val in elem.attrib.items():
            tags_with_attributes[attr].append(val)
    
    return dict(tags_with_values), dict(tags_with_attributes)

print("=== CFD Simulation Analysis ===")
tags, attrs = analyze_xml(CFD_XML)
print("Tags with values:", dict(tags))
print("Tags with attributes:", dict(attrs))

print("\n=== FEA Analysis ===")
tags, attrs = analyze_xml(FEA_XML)
print("Tags with values:", dict(tags))

print("\n=== Thermal Analysis ===")
tags, attrs = analyze_xml(THERMAL_XML)
print("Tags with values:", dict(tags))

## 4. Create Custom Tag Shadow Maps

Design custom tag mappings for each simulation type.

In [ ]:
# Define custom shadow maps for different simulation types

CFD_TAG_MAP = {
    # Physical quantities
    "velocity_magnitude": "flow_rate_metric",
    "velocity_inlet": "inlet_flow_rate",
    "pressure": "pressure_reading",
    "pressure_outlet": "outlet_pressure",
    "temperature": "thermal_reading",
    "turbulence_intensity": "turbulence_factor",
    # Domain
    "domain": "region_alpha",
    "results": "output_data",
    "point": "measurement_point",
}

FEA_TAG_MAP = {
    # Structural analysis
    "force": "load_vector",
    "magnitude": "force_magnitude",
    "displacement_x": "dx",
    "displacement_y": "dy",
    "stress_von_mises": "stress_equivalent",
    # Domain
    "loads": "applied_forces",
    "results": "structural_output",
    "node": "grid_point",
}

THERMAL_TAG_MAP = {
    # Thermal quantities
    "heat_flux": "thermal_input",
    "ambient_temp": "surrounding_temp",
    "convection_coeff": "heat_transfer_factor",
    "temperature": "thermal_reading",
    "heat_generation": "power_dissipation",
    "thermal_resistance": "thermal_impedance",
    # Domain
    "boundary_conditions": "thermal_constraints",
    "results": "thermal_output",
    "component": "element_node",
}

print("=== CFD Tag Map ===")
for orig, shadow in CFD_TAG_MAP.items():
    print(f"  {orig:25} → {shadow}")

print("\n=== FEA Tag Map ===")
for orig, shadow in FEA_TAG_MAP.items():
    print(f"  {orig:25} → {shadow}")

print("\n=== Thermal Tag Map ===")
for orig, shadow in THERMAL_TAG_MAP.items():
    print(f"  {orig:25} → {shadow}")

## 5. Generate Policies

Create policy files for each simulation type.

In [ ]:
from src.policy_engine import Policy, Rule
import json

def create_policy(name, tag_map, description):
    """Create a policy with rules for each tag in the map."""
    rules = []
    for tag in tag_map.keys():
        rules.append({
            "tag_pattern": tag,
            "action": "mask_value",
            "parameters": None
        })
    
    policy = {
        "version": "1.0",
        "name": name,
        "description": description,
        "global_masking": True,
        "tag_shadow_map": tag_map,
        "rules": rules
    }
    return policy

# Create policies
cfd_policy = create_policy(
    "cfd-default",
    CFD_TAG_MAP,
    "Default policy for CFD simulations"
)

fea_policy = create_policy(
    "fea-default",
    FEA_TAG_MAP,
    "Default policy for FEA analyses"
)

thermal_policy = create_policy(
    "thermal-default",
    THERMAL_TAG_MAP,
    "Default policy for thermal analyses"
)

# Save policies
os.makedirs('config', exist_ok=True)
for name, policy in [("cfd", cfd_policy), ("fea", fea_policy), ("thermal", thermal_policy)]:
    with open(f'config/policy_{name}.json', 'w') as f:
        json.dump(policy, f, indent=2)
    print(f"✓ Created config/policy_{name}.json")

## 6. Test Policies

Apply policies to sample data and verify results.

In [ ]:
from src.config import MaskingConfig, load_config
from src.gatekeeper import apply_gatekeeper, mask_values, _apply_tag_shadowing
from src.vault import Vault
from pathlib import Path
from lxml import etree

def apply_policy(xml_string, tag_map):
    """Apply a custom policy to XML."""
    config = MaskingConfig()
    vault = Vault('test_vault.json')
    
    # Parse
    tree = etree.fromstring(xml_string.encode())
    
    # Mask values
    masked_tree = mask_values(tree, config, vault)
    
    # Shadow tags
    _apply_tag_shadowing(masked_tree, tag_map)
    
    return etree.tostring(masked_tree, encoding='unicode'), vault

# Test CFD policy
print("=== CFD Policy Test ===\n")
cfd_sanitized, cfd_vault = apply_policy(CFD_XML, CFD_TAG_MAP)
print(cfd_sanitized)
print(f"\nVault entries: {len(cfd_vault)}")

# Test FEA policy
print("\n=== FEA Policy Test ===\n")
fea_sanitized, fea_vault = apply_policy(FEA_XML, FEA_TAG_MAP)
print(fea_sanitized)

## 7. Save Custom Policies

In [ ]:
# Save all policies to config directory
os.makedirs('config/policies', exist_ok=True)

policies = {
    'cfd': cfd_policy,
    'fea': fea_policy,
    'thermal': thermal_policy
}

for name, policy in policies.items():
    path = f'config/policies/{name}_policy.json'
    with open(path, 'w') as f:
        json.dump(policy, f, indent=2)
    print(f"✓ Saved: {path}")

print("\n=== Policy Summary ===")
for name, policy in policies.items():
    print(f"\n{name.upper()}:")
    print(f"  Description: {policy['description']}")
    print(f"  Tags mapped: {len(policy['tag_shadow_map'])}")
    print(f"  Rules: {len(policy['rules'])}")

## Summary

You've learned how to:

1. **Analyze XML schemas** - Identify tags and numeric values
2. **Design tag shadow maps** - Create custom mappings for different simulation types
3. **Generate policies** - Create reusable policy JSON files
4. **Test policies** - Verify sanitization works correctly
5. **Save policies** - Store for later use in production

### Next Steps
- Use these policies with the MCP server
- Add shuffle_siblings rules for structural anonymity
- Integrate with your simulation workflow